# api_dev.ipynb

In [ ]:
import json
import pandas as pd
import io, os
from utils import db
import requests

## Electricity Data

This dataset is monthly aggregated data. The function to return this data needs to be executed once a month.

In [ ]:
e_data = db.load_electricity_data('output.json')
e_data.head()

In [ ]:
import datetime

datetime.datetime.now().year
datetime.datetime.now().month

In [ ]:
#api_key = ""
state_id = "OH"
start_yr_mnth = '2018-01'
end_yr_mnth = '2023-06'
url = f"https://api.eia.gov/v2/electricity/retail-sales/data?api_key={api_key}&frequency=monthly&data[0]=customers&data[1]=price&data[2]=revenue&data[3]=sales&facets[stateid][]={state_id}&start={start_yr_mnth}&end={end_yr_mnth}&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000"

In [ ]:
resp = requests.get(url)

with open('./electric_consumption.json', 'w') as f:
    json.dump(resp.json(), f)


In [ ]:
df = pd.json_normalize(resp.json()['response']['data'])

# Weather Observation Data

In [ ]:
# Temperature data
zone_cde = 'OHZ055'
res = requests.get(f"https://api.weather.gov/zones/forecast/{zone_cde}/observations")

In [ ]:
df = pd.json_normalize(res.json(), record_path = 'features')

## Get Weather Data Saved to S3

In [ ]:
import boto3

s3 = boto3.client('s3')

fc = s3.get_object(Bucket = 'cw-weather-data',Key = "observations/OHZ055_2023-08-04.json")
file_content = fc['Body'].read().decode('utf-8')
json_content = json.loads(file_content)
json_content